In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from joblib import dump, load
from sklearn.metrics import accuracy_score, classification_report


# =================================================================
# 1. Loading the Data with Dask 
# =================================================================

def preprocess_data(api_path, dll_path, pe_path, is_training=False):
   
    api = dd.read_csv(
        api_path, 
        assume_missing=True,
        blocksize="64MB",  
        sample=1_000_000,  
    )
    dlls = dd.read_csv(dll_path, assume_missing=True)
    pe = dd.read_csv(pe_path, assume_missing=True)

    
    api_summary = (
        api.groupby("SHA256")
        .size()
        .reset_index()
        .rename(columns={0: "api_function_count"})
    )
    
    
    dlls_summary = (
        dlls.set_index("SHA256")
        .iloc[:, 2:]
        .sum(axis=1)
        .reset_index()
        .rename(columns={0: "dll_count"})
    )

    # Merge all datasets with outer join
    merged = (
        api_summary.merge(dlls_summary, on="SHA256", how="outer")
        .merge(pe, on="SHA256", how="outer")
    )

    
    merged = merged.fillna({
        'api_function_count': 0,
        'dll_count': 0
    })
    
    
    merged = merged.fillna(0)

    
    df = merged.compute()

    # Prepare features/target
    if is_training:
        y = df["Type"]
        X = df.drop(columns=["SHA256", "Type"])
        return X, y
    else:
        if "Type" in df.columns:
            df = df.drop(columns=["Type"])
        return df  


In [16]:


# =================================================================
# 2. Training Pipeline 
# =================================================================
X, y = preprocess_data(
    "/kaggle/input/mined-crest-dataset/dataset/API_Functions.csv",
    "/kaggle/input/mined-crest-dataset/dataset/DLLs_Imported.csv",
    "/kaggle/input/mined-crest-dataset/dataset/portable_executable.csv",
    is_training=True
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add imputation in pipeline 
pipeline = make_pipeline(
    SimpleImputer(strategy='constant', fill_value=0),
    RandomForestClassifier(n_estimators=100, random_state=42)
)

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))



Accuracy: 0.8892
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.94       357
         1.0       0.96      0.94      0.95       919
         2.0       0.99      0.99      0.99       889
         3.0       0.91      0.85      0.88       958
         4.0       0.93      0.90      0.92       936
         5.0       0.76      0.77      0.77       817
         6.0       0.73      0.84      0.78       728

    accuracy                           0.89      5604
   macro avg       0.89      0.89      0.89      5604
weighted avg       0.89      0.89      0.89      5604



In [2]:
dump(pipeline, "final_model.joblib")

['final_model.joblib']

In [19]:
from sklearn.model_selection import train_test_split,StratifiedKFold, cross_val_score

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 
cv_results = cross_val_score(pipeline, X, y, cv=kfold, scoring='accuracy')

print(f"Cross-Validation Accuracy Scores: {cv_results}")
print(f"Mean Accuracy: {cv_results.mean():.4f}")
print(f"Standard Deviation: {cv_results.std():.4f}")

Cross-Validation Accuracy Scores: [0.89114918 0.8914867  0.8848831  0.88434767 0.88916652]
Mean Accuracy: 0.8882
Standard Deviation: 0.0030


In [20]:
y_pred = pipeline.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f"Random Forest Accuracy: {accuracy:.4f}")


# Classification report for Precision, Recall, and F1-score
print("\nClassification Report:")
print(classification_report(y, y_pred))



Random Forest Accuracy: 0.9768

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1794
         1.0       0.99      0.99      0.99      4785
         2.0       1.00      1.00      1.00      4408
         3.0       0.98      0.97      0.97      4682
         4.0       0.99      0.98      0.98      4824
         5.0       0.95      0.95      0.95      4003
         6.0       0.93      0.97      0.95      3520

    accuracy                           0.98     28016
   macro avg       0.98      0.98      0.98     28016
weighted avg       0.98      0.98      0.98     28016

